In [8]:
import sys

sys.path.insert(0, "..")


# Topic Detection

This is an agent that will notice when the user has changed topics. The current topic is used to help other parts of the system stay on task.

In [9]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import ZeroShotAgent

input_variables = ["input", "history", "current_topic"]
template = """
Below is a chat log of a conversation between a human user an AI agent, all related to the same topic summarized in "Current Topic". "Input" is a new message from the user that may or may not change topics. Your task is to do the following:

1- Compare the Input to the current topic and previous messages to determine if the input refers to a new topic or the same one. Generate a "topic change score" that is one of the following:

1 -- input changes topic, high confidence
0.5 -- input changes topic, moderate confidence
-0.5 -- input does not change topic, moderate confidence 
-1 -- input does not change topic, high confidence

2- Generate an updated topic summary according to the following rules:
  1. If the topic has changed or the current topic is empty, create a new 5 word summary of the topic based on the input message alone.
  2. If the topic has not changed, keep the same Current Topic

Your output should be a single line of JSON with the following format:
[topic change score, "new topic summary"]

Current Topic:
{current_topic}

Chat Log:
{history}

Input:
{input}

Output: 
"""

llm = OpenAI(temperature=0)
prompt = PromptTemplate(template=template,input_variables=input_variables)
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

def to_chat_log(history):
  lines = []
  for line in history:
    sender = line["s"]
    message = line["m"]
    lines.append(f"{sender}: {message}")
  return "\n".join(lines)

def label_topic(history, idx, current_topic=""):
    message = history[idx]["m"]
    chat_log = to_chat_log(history[0:idx])
    topic = chain.run(
      input=message,
      history=chat_log,
      current_topic=current_topic)
    print(topic)
    return topic



In [13]:
from sessions import plan_date
topics = []
topic = ""
for idx in range(len(plan_date)):
  topic = label_topic(plan_date, idx, current_topic=topic)
  topics.append(topic)
topics




> Entering new LLMChain chain...
Prompt after formatting:

Below is a chat log of a conversation between a human user an AI agent, all related to the same topic summarized in "Current Topic". "Input" is a new message from the user that may or may not change topics. Your task is to do the following:

1- Compare the Input to the current topic and previous messages to determine if the input refers to a new topic or the same one. Generate a "topic change score" that is one of the following:

1 -- input changes topic, high confidence
0.5 -- input changes topic, moderate confidence
-0.5 -- input does not change topic, moderate confidence 
-1 -- input does not change topic, high confidence

2- Generate an updated topic summary according to the following rules:
  1. If the topic has changed or the current topic is empty, create a new 5 word summary of the topic based on the input message alone.
  2. If the topic has not changed, keep the same Current Topic

Your output should be a single lin

['[1, "dating plans weekend unsure"]',
 '[0.5, "date ideas suggestions"]',
 '[-0.5, "date ideas suggestions"]',
 '[-1, "date ideas suggestions"]',
 '[0.5, "vegan date ideas suggestions"]',
 '[-0.5, "vegan date ideas suggestions"]',
 '[1, "vegan restaurant date ideas"]',
 '[0.5, "vegan cooking class date ideas"]',
 '[1, "vegan food festival date ideas"]',
 '[1, "vegan snacks picnic park treats"]',
 '[-1, "vegan dinner park stroll sunset"]',
 '[1, "vegan restaurants santa monica"]',
 '[-1, "vegan restaurants santa monica"]',
 '[-0.5, "vegan restaurants santa monica"]',
 '[-1, "vegan restaurants santa monica"]',
 '[-0.5, "vegan restaurants santa monica"]',
 '[-1, "vegan restaurants santa monica"]',
 '[-0.5, "vegan restaurants santa monica"]',
 '[-1, "vegan restaurants santa monica"]',
 '[-1, "vegan restaurants santa monica"]']